In [ ]:
import numpy as np
import elements
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
cb = elements.central_body()
unit_length = 3000e3 # 10 km
cable_length = 100000e3
# Theorize there are numerical percision errors at low altitudes
phys_prop = elements.link_physical_properties(unit_length=unit_length, diamter=1, density=1300)
lengths = np.arange(cable_length, 10000, -unit_length)
links = []

for i, l in enumerate(lengths):
    links.append(elements.link(np.array([l -(unit_length/2),0,0],dtype=np.float64), central_body=cb, n=i, phys_prop=phys_prop))
    print(np.array([l -(unit_length/2),0,0]))
# plt.figure(figsize=(9,9))
# plt.axis('equal')
# th = np.linspace(0, 2 * np.pi)
# plt.plot(6378 * np.cos(th), 6378 * np.sin(th), 'g-')
# plt.plot(39000 * np.cos(th), 39000 * np.sin(th), 'r--')
# plt.plot(36000 * np.cos(th), 36000 * np.sin(th), 'b--')
# [plt.plot(link.pos_ECI[0] / 1000, link.pos_ECI[1] / 1000, 'k.') for link in links]
# # for dt in [1] * 180:
#     [link.propogate_motion(dt) for link in links]
# [plt.plot(link.pos_ECI[0] / 1000, link.pos_ECI[1] / 1000, 'g.') for link in links]


plt.figure()

anchor = elements.space_anchor(np.array([cable_length,0,0],dtype=np.float64), central_body=cb, mass=0e3)
anchor.calculate_tension()
links.insert(0, anchor)
tensions = [np.linalg.norm(anchor.tension)]
radius = [np.linalg.norm(anchor.pos_ECI)/ 42300E3]
for i, link in enumerate(links[1::]):
    link.calculate_tension(links[i].tension)
    tensions.append(np.linalg.norm(link.tension))
    radius.append(np.linalg.norm(link.pos_ECI) / 42300E3)
plt.plot(radius, tensions, '.')


# plt.figure()
# R = np.linspace(6378e3,cable_length)
# T = cb.mu * phys_prop.density * ( (1/6378e3) - (3 / (2 * R)) + ((6378e3 ** 2) / (2 * R **3 )))
# plt.plot(R / 36000e3,T)


In [ ]:
pass
link1 = links[1]
print(link1.calculate_gravity_force(link1.phys_prop.mass) / link1.phys_prop.mass)
print(link1.acl_ECI)


In [ ]:
link.tension

In [ ]:
link.acl_ECI

In [ ]:
link.calculate_gravity_force(mass=link.phys_prop.mass)

In [ ]:
link.pos_ECI * phys_prop.mass / link.calculate_gravity_force(mass=link.phys_prop.mass)